# [ 4-4. 현금흐름 계산4 ]
## 1. 필요한 모듈 import
* 기존 작성한 모듈 및 필요 파일 확인
    - FCHotel_FSmodeling_assumption.xlsx
    - m00_general_function.py
    - m01_assumption.py
    - m02_index.py
    - m03_funding.py
    - m04_operating_income.py
    - m05_operating_cost.py
    - m06_facility_cost.py

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', 300)

# DataFrame의 출력을 확장하여 한 줄로 계속 출력되도록 설정
pd.set_option('display.expand_frame_repr', True)

from m00_general_function import df_seperator
from m01_assumption import assumption
from m02_index import index
from m03_funding import funding
from m04_operating_income import operating_income
from m05_operating_cost import operating_cost
from m06_facility_cost import facility_cost

## 2. 차입원리금 상환 현금흐름 작성
### 2-1. funding['차입금'] 객체 확인

In [2]:
funding['차입금']

,차입금유입,차입금상환,차입이자,차입금잔액
2023-12-31,10000000000,0,0,10000000000
2024-01-31,0,0,42349730,10000000000
2024-02-29,0,0,39617490,10000000000
2024-03-31,0,50000000,42349730,9950000000
2024-04-30,0,0,40778690,9950000000
...,...,...,...,...
2026-09-30,0,50000000,39041100,9450000000
2026-10-31,0,0,40130140,9450000000
2026-11-30,0,0,38835620,9450000000
2026-12-31,0,50000000,40130140,9400000000


### 2-2. 차입금 객체의 현금흐름 반영

In [3]:
#### cashflow, balance 객체 초기화 ####
cashflow = pd.DataFrame(
    columns = ['date', 'categoryA', 'categoryB', 'categoryC', '입금금액', '출금금액']
)

balance = pd.DataFrame({
    '기초현금': [0] * len(index['model']),
    '입금금액': [0] * len(index['model']),
    '출금금액': [0] * len(index['model']),
    '기말현금': [0] * len(index['model']), 
},
    index=index['model']
)
room_type_list = list(assumption['business_overview']['객실수'].keys())


#### cashflow, balance 작성 ####
idx = 0
cash_balance = 0
for dt in index['model']:
    #### 0. 기초현금 계산
    balance.loc[dt, '기초현금'] = cash_balance

    #### 1. 자금조달소요
    ## 1-1. 자기자본 유입
    amount = funding['자기자본'].loc[dt, '자기자본유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '자기자본', '자기자본유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-2. 차입금 유입
    amount = funding['차입금'].loc[dt, '차입금유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '차입금', '차입금유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-3. 자산매입
    amount = funding['자산매입'].loc[dt, '자산매입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '자산매입', '매입대금지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = funding['자산매입'].loc[dt, '매입부수비용']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '매입부수비용', '부수비용지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 2. 운영수입
    for room_type in room_type_list:
        amount = operating_income[room_type].loc[dt, '객실수입']
        if amount > 0:
            cashflow.loc[idx] = [dt, '운영수입', '객실수입', room_type, amount, 0]
            balance.loc[dt, '입금금액'] += amount
            idx += 1

    #### 3. 운영비용 - 객실운영비
    ## 3-1. 청소세탁비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['청소세탁비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '청소세탁비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    ## 3-2. 수도광열비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['수도광열비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    amount = operating_cost['객실운영비']['수도광열비'].loc[dt, 'Overhead']
    if amount > 0:
        cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', 'Overhead', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 3-3. 예약수수료
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['예약수수료'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '예약수수료', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    #### 4. 운영비용 - 관리운영비
    ## 4-1. 광고홍보비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '광고홍보비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '광고홍보비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 4-2. 기타운영비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '기타운영비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '기타운영비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 5. 운영비용 - 인건비
    ## 5-1. 객실운영팀
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-2. 경영지원팀
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '임원']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '임원', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-3. 마케팅팀
    amount = operating_cost['인건비']['마케팅팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '마케팅팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-4. 시설관리팀
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 6. 시설관리비
    ## 6-1. 통상수선비
    for room_type in room_type_list:
        amount = facility_cost['통상수선비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '시설관리비', '통상수선비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    ## 6-2. 대수선공사비
    for room_type in room_type_list:
        amount = facility_cost['대수선공사비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '시설관리비', '대수선공사비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    #### 7. 차입원리금 상환
    ## 7-1. 차입원금 상환
    amount = funding['차입금'].loc[dt, '차입금상환']
    if amount > 0:
        cashflow.loc[idx] = [dt, '차입원리금상환', '차입금', '차입금상환', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 7-2. 차입이자 상환
    amount = funding['차입금'].loc[dt, '차입이자']
    if amount > 0:
        cashflow.loc[idx] = [dt, '차입원리금상환', '차입금', '차입이자', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1


    #### 9. 기말현금 계산
    cash_balance = (
        balance.loc[dt, '기초현금'] + balance.loc[dt, '입금금액'] - balance.loc[dt, '출금금액']
    )
    balance.loc[dt, '기말현금'] = cash_balance

In [4]:
cashflow[cashflow['categoryA']=='차입원리금상환']

,date,categoryA,categoryB,categoryC,입금금액,출금금액
39,2024-01-31,차입원리금상환,차입금,차입이자,0,42349730
65,2024-02-29,차입원리금상환,차입금,차입이자,0,39617490
91,2024-03-31,차입원리금상환,차입금,차입금상환,0,50000000
92,2024-03-31,차입원리금상환,차입금,차입이자,0,42349730
118,2024-04-30,차입원리금상환,차입금,차입이자,0,40778690
...,...,...,...,...,...,...
870,2026-09-30,차입원리금상환,차입금,차입이자,0,39041100
896,2026-10-31,차입원리금상환,차입금,차입이자,0,40130140
922,2026-11-30,차입원리금상환,차입금,차입이자,0,38835620
948,2026-12-31,차입원리금상환,차입금,차입금상환,0,50000000


## 3. 배당금 지급 현금흐름 작성
### 3-1. 배당금 계산 전 현금 잔액 계산

In [5]:
#### cashflow, balance 객체 초기화 ####
cashflow = pd.DataFrame(
    columns = ['date', 'categoryA', 'categoryB', 'categoryC', '입금금액', '출금금액']
)

balance = pd.DataFrame({
    '기초현금': [0] * len(index['model']),
    '입금금액': [0] * len(index['model']),
    '출금금액': [0] * len(index['model']),
    '기말현금': [0] * len(index['model']), 
},
    index=index['model']
)
room_type_list = list(assumption['business_overview']['객실수'].keys())


#### cashflow, balance 작성 ####
idx = 0
cash_balance = 0
for dt in index['model']:
    #### 0. 기초현금 계산
    balance.loc[dt, '기초현금'] = cash_balance

    #### 1. 자금조달소요
    ## 1-1. 자기자본 유입
    amount = funding['자기자본'].loc[dt, '자기자본유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '자기자본', '자기자본유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-2. 차입금 유입
    amount = funding['차입금'].loc[dt, '차입금유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '차입금', '차입금유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-3. 자산매입
    amount = funding['자산매입'].loc[dt, '자산매입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '자산매입', '매입대금지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = funding['자산매입'].loc[dt, '매입부수비용']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '매입부수비용', '부수비용지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 2. 운영수입
    for room_type in room_type_list:
        amount = operating_income[room_type].loc[dt, '객실수입']
        if amount > 0:
            cashflow.loc[idx] = [dt, '운영수입', '객실수입', room_type, amount, 0]
            balance.loc[dt, '입금금액'] += amount
            idx += 1

    #### 3. 운영비용 - 객실운영비
    ## 3-1. 청소세탁비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['청소세탁비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '청소세탁비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    ## 3-2. 수도광열비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['수도광열비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    amount = operating_cost['객실운영비']['수도광열비'].loc[dt, 'Overhead']
    if amount > 0:
        cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', 'Overhead', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 3-3. 예약수수료
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['예약수수료'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '예약수수료', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    #### 4. 운영비용 - 관리운영비
    ## 4-1. 광고홍보비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '광고홍보비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '광고홍보비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 4-2. 기타운영비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '기타운영비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '기타운영비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 5. 운영비용 - 인건비
    ## 5-1. 객실운영팀
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-2. 경영지원팀
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '임원']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '임원', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-3. 마케팅팀
    amount = operating_cost['인건비']['마케팅팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '마케팅팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-4. 시설관리팀
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 6. 시설관리비
    ## 6-1. 통상수선비
    for room_type in room_type_list:
        amount = facility_cost['통상수선비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '시설관리비', '통상수선비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    ## 6-2. 대수선공사비
    for room_type in room_type_list:
        amount = facility_cost['대수선공사비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '시설관리비', '대수선공사비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    #### 7. 차입원리금 상환
    ## 7-1. 차입원금 상환
    amount = funding['차입금'].loc[dt, '차입금상환']
    if amount > 0:
        cashflow.loc[idx] = [dt, '차입원리금상환', '차입금', '차입금상환', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 7-2. 차입이자 상환
    amount = funding['차입금'].loc[dt, '차입이자']
    if amount > 0:
        cashflow.loc[idx] = [dt, '차입원리금상환', '차입금', '차입이자', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 8. 자기자본 배당
    ## 8-1. 배당 전 현금 잔액 계산
    cash_balance = balance.loc[dt, '기초현금'] + balance.loc[dt, '입금금액'] - balance.loc[dt, '출금금액']
    print(f"{dt}: {cash_balance}")


    #### 9. 기말현금 계산
    cash_balance = (
        balance.loc[dt, '기초현금'] + balance.loc[dt, '입금금액'] - balance.loc[dt, '출금금액']
    )
    balance.loc[dt, '기말현금'] = cash_balance

2023-12-31 00:00:00: 905871236
2024-01-31 00:00:00: 1144288339
2024-02-29 00:00:00: 1344733797
2024-03-31 00:00:00: 1425477400
2024-04-30 00:00:00: 1551466450
2024-05-31 00:00:00: 1687443803
2024-06-30 00:00:00: 1742048853
2024-07-31 00:00:00: 1990889456
2024-08-31 00:00:00: 2239730059
2024-09-30 00:00:00: 2320784029
2024-10-31 00:00:00: 2457184882
2024-11-30 00:00:00: 2562199772
2024-12-31 00:00:00: 2757777625
2025-01-31 00:00:00: 2991257469
2025-02-28 00:00:00: 3178809976
2025-03-31 00:00:00: 1975560704
2025-04-30 00:00:00: 604784793
2025-05-31 00:00:00: -338267669
2025-06-30 00:00:00: -287541820
2025-07-31 00:00:00: -43337316
2025-08-31 00:00:00: 200867188
2025-09-30 00:00:00: 277886997
2025-10-31 00:00:00: 410142795
2025-11-30 00:00:00: 511279604
2025-12-31 00:00:00: 701892178
2026-01-31 00:00:00: 930473433
2026-02-28 00:00:00: 1113561418
2026-03-31 00:00:00: 1186134228
2026-04-30 00:00:00: 1304171427
2026-05-31 00:00:00: 1431918154
2026-06-30 00:00:00: 1478828594
2026-07-31 00:00:

### 3-2. 배당금 계산 및 현금흐름에 반영

In [6]:
#### cashflow, balance 객체 초기화 ####
cashflow = pd.DataFrame(
    columns = ['date', 'categoryA', 'categoryB', 'categoryC', '입금금액', '출금금액']
)

balance = pd.DataFrame({
    '기초현금': [0] * len(index['model']),
    '입금금액': [0] * len(index['model']),
    '출금금액': [0] * len(index['model']),
    '기말현금': [0] * len(index['model']), 
},
    index=index['model']
)
room_type_list = list(assumption['business_overview']['객실수'].keys())


#### cashflow, balance 작성 ####
idx = 0
cash_balance = 0
for dt in index['model']:
    #### 0. 기초현금 계산
    balance.loc[dt, '기초현금'] = cash_balance

    #### 1. 자금조달소요
    ## 1-1. 자기자본 유입
    amount = funding['자기자본'].loc[dt, '자기자본유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '자기자본', '자기자본유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-2. 차입금 유입
    amount = funding['차입금'].loc[dt, '차입금유입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자금조달', '차입금', '차입금유입', amount, 0]
        balance.loc[dt, '입금금액'] += amount
        idx += 1

    ## 1-3. 자산매입
    amount = funding['자산매입'].loc[dt, '자산매입']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '자산매입', '매입대금지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = funding['자산매입'].loc[dt, '매입부수비용']
    if amount > 0:
        cashflow.loc[idx] = [dt, '자산매입', '매입부수비용', '부수비용지출', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 2. 운영수입
    for room_type in room_type_list:
        amount = operating_income[room_type].loc[dt, '객실수입']
        if amount > 0:
            cashflow.loc[idx] = [dt, '운영수입', '객실수입', room_type, amount, 0]
            balance.loc[dt, '입금금액'] += amount
            idx += 1

    #### 3. 운영비용 - 객실운영비
    ## 3-1. 청소세탁비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['청소세탁비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '청소세탁비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    ## 3-2. 수도광열비
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['수도광열비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    amount = operating_cost['객실운영비']['수도광열비'].loc[dt, 'Overhead']
    if amount > 0:
        cashflow.loc[idx] = [dt, '객실운영비', '수도광열비', 'Overhead', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 3-3. 예약수수료
    for room_type in room_type_list:
        amount = operating_cost['객실운영비']['예약수수료'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '객실운영비', '예약수수료', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1
    
    #### 4. 운영비용 - 관리운영비
    ## 4-1. 광고홍보비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '광고홍보비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '광고홍보비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 4-2. 기타운영비
    amount = operating_cost['관리운영비']['관리운영비'].loc[dt, '기타운영비']
    if amount > 0:
        cashflow.loc[idx] = [dt, '관리운영비', '관리운영비', '기타운영비', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 5. 운영비용 - 인건비
    ## 5-1. 객실운영팀
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['객실운영팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '객실운영팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-2. 경영지원팀
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['경영지원팀'].loc[dt, '임원']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '경영지원팀', '임원', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-3. 마케팅팀
    amount = operating_cost['인건비']['마케팅팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '마케팅팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    ## 5-4. 시설관리팀
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '정규직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '정규직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    amount = operating_cost['인건비']['시설관리팀'].loc[dt, '임시직']
    if amount > 0:
        cashflow.loc[idx] = [dt, '인건비', '시설관리팀', '임시직', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 6. 시설관리비
    ## 6-1. 통상수선비
    for room_type in room_type_list:
        amount = facility_cost['통상수선비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '시설관리비', '통상수선비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    ## 6-2. 대수선공사비
    for room_type in room_type_list:
        amount = facility_cost['대수선공사비'].loc[dt, room_type]
        if amount > 0:
            cashflow.loc[idx] = [dt, '시설관리비', '대수선공사비', room_type, 0, amount]
            balance.loc[dt, '출금금액'] += amount
            idx += 1

    #### 7. 차입원리금 상환
    ## 7-1. 차입원금 상환
    amount = funding['차입금'].loc[dt, '차입금상환']
    if amount > 0:
        cashflow.loc[idx] = [dt, '차입원리금상환', '차입금', '차입금상환', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1
    
    ## 7-2. 차입이자 상환
    amount = funding['차입금'].loc[dt, '차입이자']
    if amount > 0:
        cashflow.loc[idx] = [dt, '차입원리금상환', '차입금', '차입이자', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1

    #### 8. 자기자본 배당
    ## 8-1. 배당 전 현금 잔액 계산
    cash_balance = balance.loc[dt, '기초현금'] + balance.loc[dt, '입금금액'] - balance.loc[dt, '출금금액']
    
    ## 8-2. 배당금 지급
    if assumption['funding_assumptions']['equity']['최초배당일'] <= dt and dt.month == 12:
        cash_balance = max(cash_balance, 0)
        amount = int(round(cash_balance * assumption['funding_assumptions']['equity']['배당률'], -3))
        funding['자기자본'].loc[dt, '배당금지급'] = amount
        cashflow.loc[idx] = [dt, '자기자본배당', '자기자본배당', '배당금지급', 0, amount]
        balance.loc[dt, '출금금액'] += amount
        idx += 1


    #### 9. 기말현금 계산
    cash_balance = (
        balance.loc[dt, '기초현금'] + balance.loc[dt, '입금금액'] - balance.loc[dt, '출금금액']
    )
    balance.loc[dt, '기말현금'] = cash_balance

In [7]:
df_seperator(funding['자기자본'][funding['자기자본']['배당금지급'] > 0])

,자기자본유입,배당금지급
2025-12-31,0,"210,568,000"
2026-12-31,0,"669,513,000"


In [9]:
df_seperator(cashflow[cashflow['categoryA']=='자기자본배당'])

,date,categoryA,categoryB,categoryC,입금금액,출금금액
634,2025-12-31,자기자본배당,자기자본배당,배당금지급,0,"210,568,000"
951,2026-12-31,자기자본배당,자기자본배당,배당금지급,0,"669,513,000"


## 4. 최종 계산 완료된 cashflow와 balance

In [12]:
df_seperator(cashflow)

,date,categoryA,categoryB,categoryC,입금금액,출금금액
0,2023-12-31,자금조달,자기자본,자기자본유입,"10,000,000,000",0
1,2023-12-31,자금조달,차입금,차입금유입,"10,000,000,000",0
2,2023-12-31,자산매입,자산매입,매입대금지출,0,"18,000,000,000"
3,2023-12-31,자산매입,매입부수비용,부수비용지출,0,"1,000,000,000"
4,2023-12-31,객실운영비,수도광열비,Overhead,0,"3,000,000"
...,...,...,...,...,...,...
957,2027-01-31,인건비,경영지원팀,정규직,0,"10,618,434"
958,2027-01-31,인건비,경영지원팀,임원,0,"7,865,506"
959,2027-01-31,인건비,마케팅팀,정규직,0,"7,078,956"
960,2027-01-31,인건비,시설관리팀,정규직,0,"7,078,956"


In [22]:
df_seperator(cashflow[cashflow['date'].apply(lambda x: f'{x.year}-{x.month}') == '2025-1'])

,date,categoryA,categoryB,categoryC,입금금액,출금금액
330,2025-01-31,운영수입,객실수입,TypeA,"145,080,000",0
331,2025-01-31,운영수입,객실수입,TypeB,"167,400,000",0
332,2025-01-31,운영수입,객실수입,TypeC,"131,750,000",0
333,2025-01-31,객실운영비,청소세탁비,TypeA,0,"11,494,800"
334,2025-01-31,객실운영비,청소세탁비,TypeB,0,"13,793,760"
335,2025-01-31,객실운영비,청소세탁비,TypeC,0,"10,856,200"
336,2025-01-31,객실운영비,수도광열비,TypeA,0,"5,747,400"
337,2025-01-31,객실운영비,수도광열비,TypeB,0,"6,896,880"
338,2025-01-31,객실운영비,수도광열비,TypeC,0,"5,428,100"
339,2025-01-31,객실운영비,수도광열비,Overhead,0,"3,090,000"


In [13]:
df_seperator(balance)

,기초현금,입금금액,출금금액,기말현금
2023-12-31,0,"20,000,000,000","19,094,128,764","905,871,236"
2024-01-31,"905,871,236","444,230,000","205,812,897","1,144,288,339"
2024-02-29,"1,144,288,339","392,080,000","191,634,542","1,344,733,797"
2024-03-31,"1,344,733,797","305,040,000","224,296,397","1,425,477,400"
2024-04-30,"1,425,477,400","295,200,000","169,210,950","1,551,466,450"
...,...,...,...,...
2026-09-30,"1,747,490,404","301,200,000","228,155,785","1,820,534,619"
2026-10-31,"1,820,534,619","311,240,000","183,068,623","1,948,705,996"
2026-11-30,"1,948,705,996","268,800,000","171,478,600","2,046,027,396"
2026-12-31,"2,046,027,396","451,980,000","935,809,678","1,562,197,718"
